In [ ]:
# Import libraries
import pandas as pd
import sklearn

In [ ]:
# Import data
data = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/insurance.csv")
data.head()

,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520


In [ ]:
# Drop missing values
data.dropna(inplace=True)

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1338 entries, 0 to 1337
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       1338 non-null   int64  
 1   sex       1338 non-null   object 
 2   bmi       1338 non-null   float64
 3   children  1338 non-null   int64  
 4   smoker    1338 non-null   object 
 5   region    1338 non-null   object 
 6   charges   1338 non-null   float64
dtypes: float64(2), int64(2), object(3)
memory usage: 73.3+ KB


# 1. Linear Regression

In [ ]:
from sklearn import linear_model
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

In [ ]:
# Selecting features and target variable
X = data.iloc[:, :6]  # Features
y = data.iloc[:, 6]   # Target variable

In [ ]:
# Identify categorical columns
categorical_cols = [col for col in X.columns if X[col].dtype == 'object']

# Create a column transformer to apply one-hot encoding only to categorical columns
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(), categorical_cols)
    ],
    remainder='passthrough'  # Pass through any remaining columns unchanged
)

# Fit and transform the data
X_encoded = preprocessor.fit_transform(X)

# Convert the result back to a DataFrame (optional)
X_encoded_df = pd.DataFrame(X_encoded, columns=preprocessor.get_feature_names_out())

In [ ]:
# Initialize the linear regression model
linear_reg = LinearRegression()

# Fit the model to the data
linear_reg.fit(X_encoded_df, y)

LinearRegression()

In [ ]:
# The coefficients
print("Coefficients: \n", linear_reg.coef_)

Coefficients: 
 [    65.6571797     -65.6571797  -11924.26727096  11924.26727096
    587.00923503    234.0453356    -448.01281436   -373.04175627
    256.85635254    339.19345361    475.50054515]


In [ ]:
def twoSum(nums, target):
        for x in nums:
            for y in nums[nums.index(x)+1::]:
                if nums.index(x) == nums.index(y):
                    pass
                else:
                    if (x + y) != target:
                        pass
                    else:
                        return nums.index(x), nums.index(y)

In [ ]:
twoSum([3,3],6)
print(two)